In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

importing the necessary libraries 

In [7]:
movies=pd.read_csv(r'C:\Users\parih\OneDrive\Desktop\movie_rec\Data\movies11.csv')
rating=pd.read_csv(r'C:\Users\parih\OneDrive\Desktop\movie_rec\Data\ratings2.csv')

loading movies and ratings data frames

In [8]:
movies.info()
movies.head(10)
movies.drop('genres', axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [9]:
rating.info()
rating

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


I plotted a graph to analyse movie vs rating_count to filter movies with less ratings, and fixed minimum 100 ratings from users to consider it into final data frame.

In [10]:
rating_count = rating['movieId'].value_counts()
rating_count

p_movie = rating_count[rating_count>100].index
p_movie

df= rating[rating.movieId.isin(p_movie)]
#df
p_movie_df  = movies[movies['movieId'].isin(p_movie)]
p_movie_df


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
60529,203519,Fast & Furious Presents: Hobbs & Shaw (2019)
60869,204352,Ad Astra (2019)
60937,204542,It: Chapter Two (2019)
61005,204698,Joker (2019)


In [11]:
f,ax = plt.subplots(1,1,figsize=(12,4))
# ratings['rating'].plot(kind='hist')
plt.scatter(rating_count.index,rating_count,color='Blue')
plt.axhline(y=100,color='r')
plt.xlabel('MovieId')
plt.ylabel('No. of users voted')
plt.show()

C:\Users\parih\AppData\Local\Temp\ipykernel_18628\1500171870.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
movie_count=df['userId'].value_counts()
valid_user= movie_count[movie_count>100].index
df=df[df.userId.isin(valid_user)]
df

,userId,movieId,rating,timestamp
70,2,1,3.5,1141415820
71,2,62,0.5,1141417130
72,2,110,5.0,1141416589
73,2,150,4.0,1141415790
74,2,151,4.5,1141415643
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [13]:
f,ax = plt.subplots(1,1,figsize=(16,4))
plt.scatter(movie_count.index,movie_count,color='mediumseagreen')
plt.axhline(y=50,color='r')
plt.xlabel('UserId')
plt.ylabel('No. of votes by user')
plt.show()

C:\Users\parih\AppData\Local\Temp\ipykernel_18628\4057900429.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


And to filter out users, I removed users who rated less than 50 movies.

In [14]:
df1=df.copy()

In [15]:
new_df = df.pivot(index='movieId',columns='userId',values='rating')

Creating a matrix with userId as columns and movieId as rows.

In [16]:
new_df

userId,2,3,4,5,8,9,12,13,18,19,...,162519,162521,162524,162529,162532,162533,162534,162537,162538,162541
movieId,,,,,,,,,,,,,,,,,,,,,
1,3.5,4.0,3.0,4.0,4.0,NaN,4.0,4.0,3.0,NaN,...,5.0,4.0,4.5,2.0,NaN,4.5,4.0,NaN,2.0,NaN
2,NaN,NaN,NaN,NaN,NaN,5.0,2.0,NaN,NaN,3.5,...,2.0,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,4.0,NaN,2.0,NaN,1.5,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203519,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
new_df.fillna(0, inplace=True)
#new_df1=new_df.reset_index()
new_df

userId,2,3,4,5,8,9,12,13,18,19,...,162519,162521,162524,162529,162532,162533,162534,162537,162538,162541
movieId,,,,,,,,,,,,,,,,,,,,,
1,3.5,4.0,3.0,4.0,4.0,0.0,4.0,4.0,3.0,0.0,...,5.0,4.0,4.5,2.0,0.0,4.5,4.0,0.0,2.0,0.0
2,0.0,0.0,0.0,0.0,0.0,5.0,2.0,0.0,0.0,3.5,...,2.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,1.5,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203519,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Replacing NaN with 0, so that we can perform different operations on the matrix

In [18]:
movie_id_map = {movie_id: idx for idx, movie_id in enumerate(new_df.index)}
index_movie_map = {idx: movie_id for movie_id, idx in movie_id_map.items()}
reversed_mapping = {v: k for k, v in index_movie_map.items()}
#reversed_mapping[205383]

We can observe in the matrix that movieId are not continuous and while using the matrix will not consider the actual movieId, instead it consider it position.\
So we are mapping the movie_id and Index position to overcome this problem.

In [19]:
sparcity = 1-(np.count_nonzero(new_df)/float(new_df.size))
print(sparcity)

0.9694261905203682


We can observe in our matrix that there are lot of null (0) values, which affect the training of our model, and to estimate the number of null values I calculated sparcity.

In [20]:
csr_df = csr_matrix(new_df.values)

To overcome the high sparcity problem I am using CSR Matrix. 

In [21]:
csr_df

<10291x62777 sparse matrix of type '<class 'numpy.float64'>'
	with 19751846 stored elements in Compressed Sparse Row format>

In [22]:
model = NearestNeighbors(metric='cosine',algorithm='brute',n_jobs=-1,n_neighbors=15)

I am using **NearestNeighbour** model with tuning some hyperparameters to get the best output.

In [23]:
model.fit(csr_df)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=15)

Fitting csr data frame into our model.

In [24]:
val =(204698,5.22) 
movie = movies.loc[movies['movieId'] == val[0], 'title']
movie.values[0]

'Joker (2019)'

I was just verifying the output of this code, ignore this(hehe).

In [25]:
def get_movie_recommendation(movie_id, k=10):
   
        movie_idx = reversed_mapping[movie_id]
        #print(movie_idx)
        print(f"selected movie is : {p_movie_df.iloc[movie_idx]['title']}")
        distances, indices = model.kneighbors(csr_df[movie_idx], n_neighbors=k+1)
    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        #print(rec_movie_indices)
        recommend_frame = []
        
        for val in rec_movie_indices:
            id= val[0]
            recommend_frame.append({'Title':movies.iloc[val[0]]['title'],'Distance':val[1]})
        df4 = pd.DataFrame(recommend_frame,index=range(1,10+1))
        #print(df4)
        return df4 
        

This is our final function which give required output(recommend related movies)\
This takes movie_id as input and recommend k movies.\
At first this get the position of the input movie in our matrix using mapping data\
Then using model it calculates k nearest points,(with distance and movie_id of that point)\
Then we make a list of our output\
And make a dataframe consisting of movie_title and distance from input movie_id.

In [26]:
movie_list = p_movie_df[p_movie_df['title'].str.contains('Spider-Man')]
#len(movie_list): 
movie_id= movie_list.iloc[0]['movieId']
#print(movie_id)
get_movie_recommendation(movie_id)    
        


selected movie is : Spider-Man (2002)


,Title,Distance
1,No Man's Land (1987),0.344633
2,All Access (2001),0.337306
3,Lantana (2001),0.333877
4,Modesty Blaise (1966),0.331047
5,Those Who Love Me Can Take the Train (Ceux qui...,0.312661
6,Rock Star (2001),0.308118
7,Baby's Day Out (1994),0.304491
8,"Caveman's Valentine, The (2001)",0.299709
9,Down to You (2000),0.298843
10,Asoka (Ashoka the Great) (2001),0.253000


This cell can be used to get recommendation, we can change movie_name in the 1st line of this cell and get output from the built function.

In [27]:
def recommend (movie_name):
    try :   
        movie_list = p_movie_df[p_movie_df['title'].str.contains(movie_name)]
        #len(movie_list): 
        movie_id= movie_list.iloc[0]['movieId']
        #print(movie_id)
    
        result = get_movie_recommendation(movie_id)
        return result
    except IndexError as e :
        print(f"{movie_name} is not in data set, please check spelling and try again and make sure first letter is capital in your movir name")

    pass    

Above cell gives error and complete detail about error when we give wrong movie_name\
So I created another function to reduce the ugliness of this notebook when error occur.\
And we do not need to find and change movie_name in the above code, we can just write **recommend("type movie name here")** and we get our output.

In [28]:
recommend("Joker")

selected movie is : Joker (2019)


,Title,Distance
1,Côte d'Azur (Crustacés et coquillages) (2005),0.721594
2,Goal! The Dream Begins (Goal!) (2005),0.717992
3,Come Drink with Me (Da zui xia) (1966),0.715590
4,"Flowers of St. Francis (Francesco, giullare di...",0.705832
5,"Bittersweet Life, A (Dalkomhan insaeng) (2005)",0.694309
6,Hail the Conquering Hero (1944),0.692998
7,"Short Film About Love, A (Krótki film o milosc...",0.689043
8,Flight of the Phoenix (2004),0.671736
9,No Direction Home: Bob Dylan (2005),0.660533
10,Beowulf & Grendel (2005),0.582783


In [29]:
#please remove hash in below 2 lines before running this code
#movie_name = input("enter movie name")
#recommend(movie_name)

Above cell just reduce our work type recommend("movie name"), we can just run the code and it ask us for input(movie_name)\
then it gives output from the function.

In [30]:
import pickle

In [31]:
pickle.dump(p_movie_df.to_dict(),open('movies.pkl','wb'))

In [33]:
pickle.dump(reversed_mapping,open('map.pkl','wb'))

In [34]:
pickle.dump(model,open('model.pkl','wb'))

In [36]:
pickle.dump(csr_df,open('csr.pkl','wb'))